In [ ]:
# default_exp colab

# colab
> helpers for colab

In [1]:
#hide
from nbdev.showdoc import *

In [10]:
#export
from nicHelper.encrypt import encryptPasswordAes, decryptPasswordAes
from beartype import beartype
from getpass import getpass
from functools import partial
import subprocess, pickle

# save aws pw

In [15]:
@beartype
def saveAwsPw(awskey:bytes, awssecret:bytes,
              path:str = '/content/drive/MyDrive/.awskey', password:bytes = b''):
  '''
    save your aws password in your 
  '''
  # get password
  if not password:
    password = getpass('input the password').encode()
  
  # encrypt with aes
  encrypt = partial(encryptPasswordAes, password)
  cryptkey, keynonce = encrypt(awskey)
  cryptsecret, secretnonce = encrypt(awssecret)
  
  # save file as pickle
  with open (path ,'wb') as f:
    pickle.dump((cryptkey, cryptsecret, keynonce, secretnonce),f)
  

In [16]:
def test_saveAwsPw():
  # enter password == test ##
  saveAwsPw(awskey = b'test', awssecret=b'password', path = '/tmp/test', password = b'test')
  with open ('/tmp/test', 'rb') as f:
    r = pickle.load(f)
  print(r)
test_saveAwsPw()

(b'\xbd*o\xf3', b'\xcd\x19\xd5B\xbc\xc8\xbcG', b'\x91`v\xbf&\x9bg\x11\x98\xfb\xb1\xce\x94\xeb\xbc\xe7', b'@\xcb\xb4\xfb\xfd\xcao\xaf\xac\x04F\x17\xafMs#')


# load aws pw

In [18]:
@beartype
def loadAwsPw(path = '/content/drive/MyDrive/.awskey', password:bytes = b'', profile:str = 'test'):
  
  with open (path, 'rb') as f:
    cryptkey, cryptsecret, keynonce, secretnonce = pickle.load(f)
  decryptedKey = decryptPasswordAes(cryptkey,password,keynonce)
  decryptedSecret = decryptPasswordAes(cryptsecret, password, secretnonce)
  print( decryptedKey, decryptedSecret)
  

In [20]:
def test_loadAwsPw():
  loadAwsPw(path = '/tmp/test', password = b'test')
test_loadAwsPw()

b'test' b'password'


# set up aws

In [29]:
@beartype
def setUpAws(awskey:str, awsSecret:str, profile:str = '', region:str = 'ap-southeast-1'):
  if profile:  profileParameter = f'--profile {profile}'
  else: profileParameter = ''
    
  setupPrefix = f'aws configure {profileParameter}'
  setupKey = f'{setupPrefix} set aws_access_key_id {awskey}'
  setupSecret = f'{setupPrefix} set aws_secret_access_key {awsSecret}'
  setupRegion = f'{setupPrefix} set default.region {region}'
  
  print(f'{setupKey} \n {setupSecret}, \n{setupRegion}')
  

In [30]:
def testSetUpAws():
  setUpAws('test', 'secret', 'testProfile')
testSetUpAws()

aws configure --profile testProfile set aws_access_key_id test 
 aws configure --profile testProfile set aws_secret_access_key secret, 
aws configure --profile testProfile set default.region ap-southeast-1
